In [56]:
import csv
import json
import requests
#import elasticsearch
from elasticsearch import Elasticsearch as ES
from elasticsearch import helpers

ESURL='http://localhost:9200'
ESIDX='hai-idx'
CURL="curl --silent -H Content-Type:application/json"

# es_client = elasticsearch.Elasticsearch(ES)
es_client = ES(ESURL)
# print(es_client)
print(es_client.indices.get_alias().keys())

def postHaiIdx(row) :
    es_client.create(index=ESIDX,doc_type="doc", id=row["id"],
                    body={
                        'indx': row["id"],
                        'intent' : row["intent"],
                        'utterance' : row["utterance"]
                    })
    print("don postHaiIdx")
    
    
def makeHaiIdx() :
    #es_client.indices.delete(index="hai-idx")
    
    csv_file = open('./data/hai_data.csv', 'r', encoding='utf-8')
    '''
    rdr = csv.reader(csv_file)

    doc_ids = []
    intents = []
    utterances = []


    for doc_id,intent,utterance in rdr:
        doc_ids.append(doc_id)
        intents.append(intent)
        utterances.append(utterance)

    doc_ids.pop(0)
    intents.pop(0)
    utterances.pop(0)
    print(intents)
    '''


    #        '_id': str(row["id"]),
    reader = csv.DictReader(csv_file)
    totalrows = 0
    docs = []
    for row in reader:
        '''
        docs.append({
            '_index': ESIDX,
            '_type': 'doc',
            '_id': str(row["id"]),
            '_source': {
                'indx': row["id"],
                'intent' : row["intent"],
                'utterance' : row["utterance"]
            }
        })
        '''
        postHaiIdx(row)
        #print(row["utterance"])
        totalrows += 1
    print(totalrows)
    print(len(docs))

    csv_file.close()
    #elasticsearch.helpers.bulk(es_client, docs)
    #helpers.bulk(es_client, docs)

#makeHaiIdx()

# print(es_client.indices.get_alias().keys())

#Get a document
#https://docs.objectrocket.com/elastic_python_examples.html
#http://www.pythonexample.com/search?q=elasticsearch
# http://jason-heo.github.io/elasticsearch/2016/07/16/elasticsearch-with-python.html

#get_doc = es_client.get(index=ESIDX, doc_type='doc', id='2322')
#print(get_doc)
#print("\n")

# Search (DSL)
srch_docs = es_client.search(index = ESIDX, doc_type = 'doc',
                       body = {
                           'query': {
                               'match': {
                                   #'utterance': '컨설팅팀 알려'
                                   'utterance': {
                                       'query': '통제예산 알려', 'operator': 'and'
                                   }
                               }
                           }
                       })
#print("len : {}".format(len(srch_docs["hits"])))
#print("result count : {}".format(srch_docs["hits"]["total"]))

if srch_docs is not None :
    print("result max:{}, score:{}, intent:{}".format(
                 srch_docs["hits"]["max_score"]
                ,srch_docs["hits"]["hits"][0]["_score"]
                ,srch_docs["hits"]["hits"][0]["_source"]["intent"]))
        
#print(srch_docs)
#print(json.dumps(srch_docs, indent = 4, ensure_ascii=False))
print(json.dumps(srch_docs["hits"]["hits"][0], indent = 4, ensure_ascii=False))
#print(json.dumps(srch_docs))

#Delete a document
#es_client.delete(index=ESIDX, doc_type='doc', id=3000)



dict_keys(['.monitoring-es-6-2018.01.17', 'hai-idx', 'seunjeon-idx', '.monitoring-es-6-2018.01.19', '.monitoring-es-6-2018.01.18'])
result max:6.983786, score:6.983786, intent:ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET
{
    "_index": "hai-idx",
    "_type": "doc",
    "_id": "2319",
    "_score": 6.983786,
    "_source": {
        "indx": "2319",
        "intent": "ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET",
        "utterance": "Biz컨설팅팀 통제 예산 알려줘"
    }
}
